In [6]:

from pathlib import Path
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import shutil
import pandas as pd
from tqdm import tqdm
import time

notebooks_path = Path.cwd()
repo_path = notebooks_path.parent
print(f'current directory is: {notebooks_path}')

import utils_ric as utils
from info import patient

#get all paths for file in the result folder
result_path  = repo_path / 'data/results'
result_paths = list(result_path.glob('**/*.csv'))
comparison_df = pd.DataFrame(columns=['method', 'pat_mean', 'pat_std', 'time'])
for i in result_paths:
    #read the csv file
    df = pd.read_csv(i)
    #get the method name
    method = i.stem
    #get mean_tre column and get the mean
    mean_tre = df['mean_tre'].mean()
    #same for std
    std_tre = df['std_tre'].mean()
    #and for time
    time = df['time'].mean()
    #concate values in comparison using concat
    comparison_df = pd.concat([comparison_df, pd.DataFrame([[method, mean_tre, std_tre, time]], columns=['method', 'pat_mean', 'pat_std', 'time'])], ignore_index=True)
#save a xlsx file with the comparison
comparison_df.to_excel(repo_path / 'data/results/comparison.xlsx', index=False)


current directory is: d:\VS_Projects\MIRA_FINAL_PROJECT\MIRA_FINAL_PROJECT\notebooks
